In [ ]:
import sys
print(sys.executable)

!{sys.executable} -m pip install matplotlib --upgrade --quiet
!{sys.executable} -m pip install torch --upgrade --quiet
!{sys.executable} -m pip install torchvision --upgrade --quiet
!{sys.executable} -m pip install tqdm --upgrade --quiet
!{sys.executable} -m pip install zetascale --upgrade --quiet
!{sys.executable} -m pip install datasets

In [3]:
import torch
from torch import nn, Tensor
from zeta import SSM
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt

# download CIFAR10 dataset

In [ ]:
mean, std = [0.4914, 0.4821, 0.4465], [0.2471, 0.2435, 0.2616]

transform_train = transforms.Compose([
transforms.RandomCrop(32, padding=2),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(5),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean,
                         std=std),
    transforms.RandomErasing(p=0.75,
                             scale=(0.01, 0.3),
                             ratio=(1.0, 1.0),
                             value=0,
                             inplace =True)])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

train_set = datasets.CIFAR10(root='./datasets', train=True, download=True, transform=transform_train)
test_set = datasets.CIFAR10(root='./datasets', train=False, download=True, transform=transform_test)

train_loader = DataLoader(train_set, shuffle=True, batch_size=128, pin_memory=True, num_workers=10)
test_loader = DataLoader(test_set, shuffle=False, batch_size=128, pin_memory=True, num_workers=10)

In [5]:
# Build patcher
class PatchEmbedding(nn.Module):
    def __init__(self,
                 height:int,
                 width:int,
                 patch_size:int,
                 dim:int,
                 in_channels):
        super().__init__()

        # takes in (b, c, h, w) -> ...
        self.conv = nn.Conv2d(
            in_channels=in_channels,
            out_channels=dim, # dim = out_channels == 1 so later we can use torch.squeeze(-1)
            kernel_size=patch_size,
            stride=patch_size
        )
    
    def forward(self, x):
        out = self.conv(x) # (b, c, h, w) -> (b, dim, h', w') where h' * w' = L
        out = out.flatten(2) # (b, d, L)
        out = out.transpose(1, 2) # (b, L, d) ~ (batch_size, L, dim) with dim =1
        return out

In [6]:
# testcase
# Define parameters for CIFAR-10
batch_size = 2  # Example batch size
image_height, image_width = 32, 32  # CIFAR-10 image size
patch_size = 4  # Patch size (each patch will be 4x4 pixels)
dim = 64  # Embedding dimension (output channels of Conv2D)
in_channels = 3  # RGB images

# Create an instance of PatchEmbedding
patch_embed = PatchEmbedding(image_height, image_width, patch_size, dim, in_channels)

# Create a random input tensor representing a batch of CIFAR-10 images
input_tensor = torch.randn(batch_size, in_channels, image_height, image_width)  # (b, c, h, w)

# Forward pass through the PatchEmbedding layer
output_tensor = patch_embed(input_tensor)

# Print input and output shapes
print("Input Size:", input_tensor.shape)  # Expected: (batch_size, in_channels, height, width)
print("Output Size:", output_tensor.shape)  # Expected: (batch_size, L, dim) where L = (height/patch_size) * (width/patch_size)

Input Size: torch.Size([2, 3, 32, 32])
Output Size: torch.Size([2, 64, 64])


In [7]:
# build Visioncobra
class BiCobraBlock(nn.Module):
    def __init__(self,
                 dim: int,
                 dt_rank: int,
                 dim_inner: int,
                 d_state: int,
                 channels: int = 64):
        """
        Arguments:
            dim (int):         Embedding dimension (D). Each token is represented by a vector of this size.
            dt_rank (int):     Rank used to parameterize delta time (dt) in the SSM. Controls temporal flexibility.
            dim_inner (int):   Internal hidden size of the SSM block. Like FFN hidden size in Transformers.
            d_state (int):     Size of the hidden state vector used inside the SSM. Controls memory depth.
            channels (int):    Number of channels for convolution (used in 1D Conv). Usually set to dim.
        """
        super().__init__()

        # linear projection layer: maps input featues to dim
        self.proj = nn.Linear(dim, dim)

        # 1D convolution along sequence axis: expects input (b, c, L)
        self.conv_for = nn.Conv1d(
            channels,
            channels,
            kernel_size=3,
            padding=1,
            dilation=1
        )
        self.conv_bac = nn.Conv1d(
            channels,
            channels,
            kernel_size=3,
            padding=1,
            dilation=1
        )

        # ssm: learns long-range sequence dynmaics
        self.ssm_for = SSM(
            dim, # in_features
            dt_rank,
            dim_inner,
            d_state
        )
        self.ssm_bac = SSM(
            dim, # in_features
            dt_rank,
            dim_inner,
            d_state
        )

        # activation function (swish / SILU)
        self.swish = nn.SiLU()

    def forward(self, x: Tensor):
        """
        Input:
            x: Tensor of shape (B, L, D) — batch of sequences with D-dimensional embeddings
        Output:
            Tensor of shape (B, L, D) — processed output after convolution, SSM, and gating
        """

        skip = x

        # project input into two branches
        x_one_for = self.proj(x) # (b, L, d)
        x_one_bac = self.proj(x.flip(0)) # (b, L, d)
        x_two = self.proj(x) # (b, L, d)

        # prepare for convolution (B, L, D) -> (B, D, L)
        x_one_for = x_one_for.transpose(1, 2) 
        x_one_for = self.conv_for(x_one_for) # (b, d, L)
        x_one_for = x_one_for.transpose(1, 2) # (b, L, d)

        x_one_bac = x_one_bac.transpose(1, 2) 
        x_one_bac = self.conv_bac(x_one_bac) # (b, d, L)
        x_one_bac = x_one_bac.transpose(1, 2) # (b, L, d)

        # apply activation and SSM to first branch
        x_one_for = self.swish(x_one_for)
        x_one_for = self.ssm_for(x_one_for)

        x_one_bac = self.swish(x_one_bac)
        x_one_bac = self.ssm_bac(x_one_bac)

        # apply activation to second branch
        x_two = self.swish(x_two)

        # combine both branches element-wise
        out_for = x_one_for * x_two # (b, l, d)
        out_bac = x_one_bac * x_two # (b, l, d)

        # residual connection
        out = out_for + out_bac + skip # (b, l, d)

        return self.proj(out) # (b, l, d)

class VisionCobra(nn.Module):
    def __init__(self,
                 dim: int,
                 dt_rank: int,
                 dim_inner,
                 d_state: int,
                 channels: int,
                 depth: int, # n_layer
                 height: int,
                 width: int,
                 patch_size: int,
                 n_classes: int,
                 *args,
                 **kwargs):
        
        """
        Arguments:
            dim (int):         Embedding dimension (D) for each token.
            dt_rank (int):     Delta-time rank in the SSM for modeling temporal dynamics.
            dim_inner (int):   Hidden dimension inside the SSM block.
            d_state (int):     Size of the hidden state vector used by the SSM.
            channels (int):    Channel size for convolution layers; typically set equal to dim.
            num_tokens (int):  Size of the vocabulary (for example, 256 if input is quantized grayscale).
            depth (int):       Number of CobraBlocks (layers) stacked in the model.
        """

        super().__init__()

        assert (height % patch_size == 0 and width % patch_size == 0 ), "Invalid Patch Entry"

        self.patch_size = patch_size
        self.to_patch_embedding = PatchEmbedding(
            height,
            width,
            self.patch_size,
            dim,
            in_channels=3
        )

        # call out cobra block
        self.backbone_layers = nn.ModuleList([
                BiCobraBlock(
                dim=dim,
                dt_rank=dt_rank,
                dim_inner=dim_inner,
                d_state=d_state,
                channels=channels,
                *args,
                **kwargs
            ) for _ in range(depth)
        ])

        # cls token
        self.cls_token = nn.Parameter(torch.zeros(1, 1, dim), requires_grad=True)
        self.head = nn.Linear(dim, n_classes)
        self.n_patches = (height // patch_size) * (width // patch_size)
        self.pos_embed = nn.Parameter(torch.zeros(1, self.n_patches + 1, dim))
    
    def forward(self, x):
        #input b, l, d
        b_size = x.shape[0]
        x = self.to_patch_embedding(x)
        cls_token = self.cls_token.expand(b_size, -1, -1)
        x = torch.cat((x, cls_token), dim=1)
        x = x + self.pos_embed
        
        for layer in self.backbone_layers:
            x = layer(x)

        cls_token_end = x[:, -1]
        pred = self.head(cls_token_end)
        return pred
        
    
        

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = VisionCobra(
    height=32,
    width=32,
    patch_size=16,
    dim=256,
    n_classes=10,
    channels=256,
    d_state=64,
    dt_rank=8,
    dim_inner=256,
    depth=12
).to(device)

opt = torch.optim.Adam(model.parameters(), lr=1e-4)
loss_fn = nn.CrossEntropyLoss()

In [9]:
def evaluate(model, testloader, loss_fn):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(testloader):

            # move input and labels into device
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = loss_fn(outputs, labels)

            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    test_loss = test_loss / len(testloader)
    return test_loss, accuracy

In [10]:
# some parameter
train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []
max_epoch = 50

In [11]:
# train
for epoch in range(max_epoch):
    running_loss = 0.0
    running_correct = 0   # to track number of correct predictions
    total = 0             # to track total number of samples

    # batch-size = 128
    for i, (inputs, labels) in tqdm(enumerate(train_loader, 0)):
        # Move inputs and labels to the device
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        opt.zero_grad()

        # Forward pass
        outputs = model(inputs)

        loss = loss_fn(outputs, labels)

        running_loss += loss.item()

        # Determine class predictions and track accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        running_correct += (predicted == labels).sum().item()

        # Backward pass and optimization
        loss.backward()
        opt.step()

    epoch_accuracy = 100 * running_correct / total
    epoch_loss = running_loss / (i + 1)
    test_loss, test_accuracy = evaluate(model, test_loader, loss_fn)
    print(f"Epoch [{epoch + 1}/{max_epoch}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")

    # save for plot
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_accuracy)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)


0it [00:00, ?it/s]


1it [00:01,  1.49s/it]


2it [00:01,  1.42it/s]


3it [00:01,  2.03it/s]


4it [00:02,  2.55it/s]

...


 95%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋       | 75/79 [00:05<00:00, 15.02it/s]


 97%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎   | 77/79 [00:05<00:00, 15.00it/s]


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [00:05<00:00, 16.17it/s]


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [00:05<00:00, 13.46it/s]

Epoch [50/50], Loss: nan, Accuracy: 10.00%, Test Loss: nan, Test Accuracy: 10.00%
